In [65]:
%%writefile mincost.dat
# Nodes set
set N := Berlin Leipzig Munic Warschau Budapest Leningrad Moscow Stalingrad;
# Arcs set
set A := (Berlin, Warschau) (Berlin,Budapest) (Leipzig, Warschau) (Leipzig, Budapest) (Munic, Warschau) (Munic, Budapest) (Warschau, Leningrad) (Warschau, Moscow) (Warschau, Stalingrad) (Budapest, Leningrad) (Budapest, Moscow) (Budapest, Stalingrad);

param origin := Berlin Leipzig Munic;

#param destination := Leningrad Moscow Stalingrad;

#param transition := Warschau Budapest;

param: demand:=
Leningrad 20
Moscow 10
Stalingrad 10;

param: costs:= 
Berlin Warschau 1
Berlin Budapest 1
Leipzig Warschau 1
Leipzig Budapest 1
Munic Warschau 1
Munic Budapest 1
Warschau Leningrad 3
Warschau Moscow 4
Warschau Stalingrad 5
Budapest Leningrad 4.5
Budapest Moscow 3.5
Budapest Stalingrad 3;

param: capacity := 
Berlin Warschau 40
Berlin Budapest 40
Leipzig Warschau 40
Leipzig Budapest 40
Munic Warschau 40
Munic Budapest 40
Warschau Leningrad 40
Warschau Moscow 40
Warschau Stalingrad 40
Budapest Leningrad 40
Budapest Moscow 40
Budapest Stalingrad 40;

Overwriting mincost.dat


In [66]:
%%writefile mincost.py
from pyomo.core import Constraint
from pyomo.environ import *

# Create an instance of the model
model = AbstractModel()

# Set Nodes list
model.N = Set()
# Set Arcs
model.A = Set(within = model.N * model.N)

# Set origin
model.origin = Param(within = model.N)
# Set destination
#model.destination = Param(model.N)
# Set transition
#model.transition = Param(model.A)
# Set costs
model.costs = Param(model.A)
# Set capacity
model.capacity = Param(model.A)
# Set demand
model.demand = Param(model.N)

# Set the decission variables => number of flows
model.flow = Var(model.A,within=NonNegativeReals)

# Define objective function
def minCost_rule(model):
    return sum((model.flow[i,j] * model.costs[i,j]) for (i,j) in model.A)
model.minCost = Objective(rule=minCost_rule, sense=minimize)

#Define minimum demand constraint
def demand_rule(model,i):
    if i == "Leningrad" or i == "Moscow" or i == "Stalingrad":
        return sum(model.flow[s,d] for (s,d) in model.A if d==i)>=model.demand[i]
    else: 
        return Constraint.Skip
model.demandConstraint = Constraint(model.N, rule=demand_rule)

# Create the capacity constraints
def capacity_rule(model,i,j):
    '''
    Defines the capacity constraints, which ensure that the flow on each arc does not exceed the capacity of that arc. This constraint is defined over the set of arcs (model.A) using the Constraint class
    '''
    return model.flow[i,j] <= model.capacity[i,j]

model.capacityConstraint = Constraint(model.A, rule=capacity_rule)


# Create the flow conservation constraints
def conservation_rule(model, node):
    '''
    Defines the flow conservation constraints, which ensure that flow is conserved at each node. This constraint is defined over the set of nodes (model.N) using the Constraint class.
    '''
    if node == "Berlin" or node == "Laipzig" or node == "Munic":
        return Constraint.Skip
    if node == "Leningrad" or node == "Moscow" or node == "Stalingrad":
        #return sum(model.flow[arc] for arc in model.A if arc[1] == node) == 1
        return Constraint.Skip
    
    incoming = sum(model.flow[i,j] for (i,j) in model.A if node==i) # in
    outgoing = sum(model.flow[i,j] for (i,j) in model.A if node==j) # out
    return incoming == outgoing

model.conservationConstraint = Constraint(model.N, rule=conservation_rule)

Overwriting mincost.py


In [67]:
!pyomo solve --solver=glpk mincost.py mincost.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.01] Creating model
[    0.03] Applying solver
[    0.11] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 165.0
    Solver results file: results.yml
[    0.11] Applying Pyomo postprocessing actions
[    0.11] Pyomo Finished
